In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import pandas as pd
import numpy  as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import pickle

In [19]:

training_documents=pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/train.tsv',delimiter="\t")
testing_documents=pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/test.tsv',delimiter="\t")


In [20]:
training_documents

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [21]:
testing_documents

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario ."
66288,222349,11855,"A long-winded , predictable scenario"
66289,222350,11855,"A long-winded ,"
66290,222351,11855,A long-winded


In [22]:
phrase=training_documents.iloc[:,2]
test_phrase=testing_documents.iloc[:,2]
type(phrase)


pandas.core.series.Series

In [23]:
phrase

0         A series of escapades demonstrating the adage ...
1         A series of escapades demonstrating the adage ...
2                                                  A series
3                                                         A
4                                                    series
                                ...                        
156055                                            Hearst 's
156056                            forced avuncular chortles
156057                                   avuncular chortles
156058                                            avuncular
156059                                             chortles
Name: Phrase, Length: 156060, dtype: object

In [24]:
test_phrase

0        An intermittently pleasing but mostly routine ...
1        An intermittently pleasing but mostly routine ...
2                                                       An
3        intermittently pleasing but mostly routine effort
4               intermittently pleasing but mostly routine
                               ...                        
66287               A long-winded , predictable scenario .
66288                 A long-winded , predictable scenario
66289                                      A long-winded ,
66290                                        A long-winded
66291                                 predictable scenario
Name: Phrase, Length: 66292, dtype: object

In [25]:
phrase[0]


'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'

In [26]:
test_phrase[0]

'An intermittently pleasing but mostly routine effort .'

In [27]:
y=training_documents.iloc[:,3]
y

0         1
1         2
2         2
3         2
4         2
         ..
156055    2
156056    1
156057    3
156058    2
156059    2
Name: Sentiment, Length: 156060, dtype: int64

In [28]:
y=y.values
y

array([1, 2, 2, ..., 3, 2, 2])

In [29]:
def get_simple_pos(tag) :
    if tag.startswith('J') :
        return wordnet.ADJ
    elif tag.startswith('V') :
        return wordnet.VERB
    elif tag.startswith('N') :
        return wordnet.NOUN
    elif tag.startswith('R') :
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [30]:
nltk.download('stopwords')
stops = set(stopwords.words('english'))
import string
punctuations = list(string.punctuation)
stops.update(punctuations)
stops

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

In [31]:
nltk.download('punkt')
nltk.download('wordnet')
#cleaning
def clean_review(review) :
    words = word_tokenize(review)
    output_words = []
    for w in words :
        if w.lower() not in stops :
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w,pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    #print(output_words)
    return " ".join(output_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
nltk.download('averaged_perceptron_tagger')
phrase=phrase.values
phrase = [clean_review(review) for review in phrase]
phrase[0]


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


'series escapade demonstrate adage good goose also good gander occasionally amuses none amount much story'

In [34]:
count_vec = CountVectorizer(max_df = 0.80,min_df = 0.0005)
x_train = count_vec.fit_transform(phrase)

In [36]:
x_train_split,x_test_split,y_train_split,y_test_split = train_test_split(x_train,y,random_state = 0,test_size = 0.2)
x_train.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [37]:
len(count_vec.get_feature_names())

1510

In [48]:
pickle.dump(count_vec,open('count_vec.pkl','wb'))
count_vec=pickle.load(open('count_vec.pkl','rb'))

In [38]:
print(x_train_split.shape)
print(y_train_split.shape)
print(x_test_split.shape)
print(y_test_split.shape)

(124848, 1510)
(124848,)
(31212, 1510)
(31212,)


In [39]:
x_test = count_vec.transform(test_phrase)

In [40]:
#Logistic_regression
lr = LogisticRegression()
lr.fit(x_train_split,y_train_split)
print(lr.score(x_test_split,y_test_split))
y_pred1 = lr.predict(x_test)


0.6002178649237473


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [41]:
#SVC classifier
svc = SVC(kernel = 'rbf')
svc.fit(x_train_split,y_train_split)
print(svc.score(x_test_split,y_test_split))
y_pred = svc.predict(x_test)

0.6327694476483404


In [42]:
pickle.dump(svc,open('model.pkl','wb'))
model=pickle.load(open('model.pkl','rb'))

In [43]:
#random forest
rf = RandomForestClassifier()
rf.fit(x_train_split,y_train_split)
print(rf.score(x_test_split,y_test_split))
y_pred2 = rf.predict(x_test)

0.6258490324234269


In [44]:
#KNN
clf = KNeighborsClassifier()
clf.fit(x_train_split , y_train_split)
print(clf.score(x_test_split,y_test_split))

0.5979110598487761


In [45]:
#Naive_Bayes
nb = MultinomialNB()
nb.fit(x_train_split,y_train_split)
print(nb.score(x_test_split,y_test_split))
y_pred3 = nb.predict(x_test)

0.5881391772395232


In [46]:
import xgboost as xgb
#XGBoost
model=xgb.XGBClassifier()
model
model.fit(x_train_split,y_train_split)
print(model.score(x_test_split,y_test_split))
y_pred1 = model.predict(x_test)

0.5274573881840318
